<h1>Transfer Learning</h1>
Guide from: https://medium.com/@saitejaponugoti/transfer-learning-for-deep-neural-networks-using-tensorflow-d628e454e9e5

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
(train_raw, validation_raw, test_raw), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:20%]', 'train[80%:85%]', 'train[90%:95%]'],
    with_info=True,
    as_supervised=True,
    shuffle_files=True,
)

In [3]:
# any image size can be selected
IMG_SIZE = 160 # All images will be resized to 224x224x3

def image_formatting(image, label):
    # converting pixel values to float type
    image = tf.cast(image, tf.float32)
    # normalising the data to be in range of -1 tp +1 255/2 = 127.5
    image = (image/127.5) - 1
    # resizing all images to a shape of 224x*224*3
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label
    
#converting all images to same shape and formatting them for quicker training of the model
train = train_raw.map(image_formatting)
validation = validation_raw.map(image_formatting)
test = test_raw.map(image_formatting)

# select required batch_size and suffle_buffer_size
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1000

# creating suffled training batches batches and shuffling for training and 
# testing data doesn't matter as they are only used for evaluating the model 
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

# checking the batch formation
for image_batch, label_batch in train_batches.take(1):
   pass

# expected size (batch_size,160*160*3)
image_batch.shape

TensorShape([128, 160, 160, 3])

In [4]:
# defining the input image size 
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the MobileNet V2
# Choosing include_top = False , so that we can define our own classification layer
# Choosing weight = None to select a non-pretrained model
original_model = tf.keras.applications.MobileNetV2(
                                        input_shape=IMG_SHAPE,
                                        include_top=False,
                                        weights=None,
                                        )

# Checking the model summary and number of tunable prameters
# original_model.summary()

# lets observe what is the feature vector shape given by the model
# without the inbuilt classification layer
feature_batch = original_model(image_batch)

# Setting up that the model can be trained
# expected shape (batch_size, 5, 5, 1280)
original_model.trainable = True

In [11]:
# building a GlobalAveragePooling2D
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

# building a Dense output layer
prediction_layer = tf.keras.layers.Dense(1,activation = "sigmoid")
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

# Adding GlobalAveragePooling2D and Dense output layer to form a model
model_non_pretrained = tf.keras.Sequential([
  original_model,
  global_average_layer,
  prediction_layer
])

(128, 1280)
(128, 1)


In [12]:
# Compiling the mode by including optimizer, loss function and metrics 
base_learning_rate = 0.0001
model_non_pretrained.compile(
    optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=['accuracy']
    )

# Number of epochs
epochs_number = 10
# Fitting the model with train batches
history = model_non_pretrained.fit(
    train_batches,
    epochs=epochs_number ,verbose = 1,
    validation_data = validation_batches
    )

# Evaluating teh accuracy on test set batches
results = model_non_pretrained.evaluate(test_batches)
print('test loss, test acc:', results)

C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10


C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


37/37 [==============================] - 36s 713ms/step - loss: 0.6804 - accuracy: 0.5681 - val_loss: 0.6933 - val_accuracy: 0.4815
Epoch 2/10
37/37 [==============================] - 25s 660ms/step - loss: 0.6379 - accuracy: 0.6316 - val_loss: 0.6934 - val_accuracy: 0.4815
Epoch 3/10
37/37 [==============================] - 29s 759ms/step - loss: 0.5878 - accuracy: 0.6883 - val_loss: 0.6939 - val_accuracy: 0.4815
Epoch 4/10
37/37 [==============================] - 28s 729ms/step - loss: 0.5371 - accuracy: 0.7330 - val_loss: 0.6946 - val_accuracy: 0.4815
Epoch 5/10
37/37 [==============================] - 27s 712ms/step - loss: 0.4593 - accuracy: 0.7784 - val_loss: 0.6955 - val_accuracy: 0.4815
Epoch 6/10
37/37 [==============================] - 27s 706ms/step - loss: 0.3979 - accuracy: 0.8160 - val_loss: 0.6963 - val_accuracy: 0.4815
Epoch 7/10
37/37 [==============================] - 30s 783ms/step - loss: 0.3413 - accuracy: 0.8504 - val_loss: 0.7002 - val_accuracy: 0.4815
Epoch 8/10

In [25]:
# Create the base model from the pre-trained model MobileNet V2
frozen_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )

# lets observe what is the feature vector shape given by the model
# without the inbuilt classification layer
feature_batch = frozen_model(image_batch)
# print(feature_batch.shape)

# Freezing all the layers of the model
frozen_model.trainable = False

# Checking the number of trainable parametres Expected : 0
# frozen_model.summary()

# Adding GlobalAveragePooling2D and Dense output layer to form a model
model_frozen_pretrained = tf.keras.Sequential([
  frozen_model,
  global_average_layer,
  prediction_layer
])

# Compiling the mode by including optimizer, loss function and metrics 
model_frozen_pretrained.compile(
    optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=['accuracy']
    )

# Number of epochs
epochs_number = 10
# Fitting the model with train batches
history_unfreeze = model_frozen_pretrained.fit(
    train_batches,
    epochs=epochs_number ,verbose = 1,
    validation_data = validation_batches
    )

# Evaluating the accuracy on test set batches
results = model_frozen_pretrained.evaluate(test_batches)
print('test loss, test acc:', results)

9412608/9406464 [==============================] - 1s 0us/step


C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10


C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


37/37 [==============================] - 11s 237ms/step - loss: 0.5164 - accuracy: 0.7732 - val_loss: 0.4183 - val_accuracy: 0.8461
Epoch 2/10
37/37 [==============================] - 9s 220ms/step - loss: 0.3737 - accuracy: 0.8792 - val_loss: 0.3135 - val_accuracy: 0.9097
Epoch 3/10
37/37 [==============================] - 9s 226ms/step - loss: 0.2833 - accuracy: 0.9211 - val_loss: 0.2407 - val_accuracy: 0.9381
Epoch 4/10
37/37 [==============================] - 9s 232ms/step - loss: 0.2209 - accuracy: 0.9428 - val_loss: 0.1911 - val_accuracy: 0.9493
Epoch 5/10
37/37 [==============================] - 9s 232ms/step - loss: 0.1778 - accuracy: 0.9568 - val_loss: 0.1566 - val_accuracy: 0.9587
Epoch 6/10
37/37 [==============================] - 9s 232ms/step - loss: 0.1479 - accuracy: 0.9637 - val_loss: 0.1322 - val_accuracy: 0.9656
Epoch 7/10
37/37 [==============================] - 9s 227ms/step - loss: 0.1266 - accuracy: 0.9697 - val_loss: 0.1148 - val_accuracy: 0.9690
Epoch 8/10
37/37

In [5]:
# building a GlobalAveragePooling2D
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

# building a Dense output layer
prediction_layer = tf.keras.layers.Dense(1,activation = "sigmoid")
prediction_batch = prediction_layer(feature_batch_average)
base_learning_rate = 0.0001
# Create the base model from the pre-trained model MobileNet V2
unfrozen_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
    )
# lets observe what is the feature vector shape given by the model
# without the inbuilt classification layer
feature_batch = unfrozen_model(image_batch)
# print(feature_batch.shape)

# Freezing all the layers of the model below 100th layer
freeze_till_layer = 100

# Freeze all the layers before the `ffreeze_till_layer` layer
for layer in unfrozen_model.layers[:freeze_till_layer]:
  layer.trainable =  False

# Adding GlobalAveragePooling2D and Dense output layer to form a model
model_unfrozen_pretrained = tf.keras.Sequential([
  unfrozen_model,
  global_average_layer,
  prediction_layer
])


# Compiling the mode by including optimizer, loss function and metrics 
model_unfrozen_pretrained.compile(
    optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=['accuracy']
    )

# Number of epochs
epochs_number = 10
# Fitting the model with train batches
history_unfreeze = model_unfrozen_pretrained.fit(
    train_batches,
    epochs=epochs_number ,verbose = 1,
    validation_data = validation_batches
    )

# Evaluating the accuracy on test set batches
results = model_unfrozen_pretrained.evaluate(test_batches)
print('test loss, test acc:', results)

C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/10


C:\Users\Niklas\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\backend.py:5016: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


37/37 [==============================] - 17s 308ms/step - loss: 0.1280 - accuracy: 0.9506 - val_loss: 0.0660 - val_accuracy: 0.9785
Epoch 2/10
37/37 [==============================] - 11s 271ms/step - loss: 0.0197 - accuracy: 0.9955 - val_loss: 0.0798 - val_accuracy: 0.9768
Epoch 3/10
37/37 [==============================] - 11s 287ms/step - loss: 0.0060 - accuracy: 0.9989 - val_loss: 0.0737 - val_accuracy: 0.9759
Epoch 4/10
37/37 [==============================] - 11s 281ms/step - loss: 0.0022 - accuracy: 0.9996 - val_loss: 0.0858 - val_accuracy: 0.9768
Epoch 5/10
37/37 [==============================] - 11s 280ms/step - loss: 0.0018 - accuracy: 0.9994 - val_loss: 0.0772 - val_accuracy: 0.9828
Epoch 6/10
37/37 [==============================] - 11s 278ms/step - loss: 7.3456e-04 - accuracy: 1.0000 - val_loss: 0.0783 - val_accuracy: 0.9802
Epoch 7/10
37/37 [==============================] - 11s 277ms/step - loss: 2.4960e-04 - accuracy: 1.0000 - val_loss: 0.0896 - val_accuracy: 0.9802
Ep